In [1]:
from keras import backend as K
from sklearn.preprocessing import MultiLabelBinarizer
import pandas as pd
import jieba

from m import f1_for_car, BOW, BasicModule
import gensim
from gensim.models import Word2Vec

Using TensorFlow backend.


# capsule.py

In [ ]:
# 构建模型 capsule.py
# from capsule import *

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
from keras.layers import Dense,Input,LSTM,Bidirectional,Activation,Conv1D,GRU
from keras.callbacks import Callback
from keras.layers import Dropout,Embedding,GlobalMaxPooling1D, MaxPooling1D, Add, Flatten
from keras.preprocessing import text, sequence
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.models import Model
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from keras.layers import K, Activation
from keras.engine import Layer
from keras.layers import Dense, Input, Embedding, Dropout, Bidirectional, GRU, Flatten, SpatialDropout1D
K.clear_session()
gru_len = 128
Routings = 5
Num_capsule = 10
Dim_capsule = 16
dropout_p = 0.25
rate_drop_dense = 0.28

K.clear_session()
gru_len = 128
Routings = 5
Num_capsule = 10
Dim_capsule = 16
dropout_p = 0.25
rate_drop_dense = 0.28


def squash(x, axis=-1):
    # s_squared_norm is really small
    # s_squared_norm = K.sum(K.square(x), axis, keepdims=True) + K.epsilon()
    # scale = K.sqrt(s_squared_norm)/ (0.5 + s_squared_norm)
    # return scale * x
    s_squared_norm = K.sum(K.square(x), axis, keepdims=True)
    scale = K.sqrt(s_squared_norm + K.epsilon())
    return x / scale


# A Capsule Implement with Pure Keras
class Capsule(Layer):
    def __init__(self, num_capsule, dim_capsule, routings=3, kernel_size=(9, 1), share_weights=True,
                 activation='default', **kwargs):
        super(Capsule, self).__init__(**kwargs)
        self.num_capsule = num_capsule
        self.dim_capsule = dim_capsule
        self.routings = routings
        self.kernel_size = kernel_size
        self.share_weights = share_weights
        if activation == 'default':
            self.activation = squash
        else:
            self.activation = Activation(activation)

    def build(self, input_shape):
        super(Capsule, self).build(input_shape)
        input_dim_capsule = input_shape[-1]
        if self.share_weights:
            self.W = self.add_weight(name='capsule_kernel',
                                     shape=(1, input_dim_capsule,
                                            self.num_capsule * self.dim_capsule),
                                     # shape=self.kernel_size,
                                     initializer='glorot_uniform',
                                     trainable=True)
        else:
            input_num_capsule = input_shape[-2]
            self.W = self.add_weight(name='capsule_kernel',
                                     shape=(input_num_capsule,
                                            input_dim_capsule,
                                            self.num_capsule * self.dim_capsule),
                                     initializer='glorot_uniform',
                                     trainable=True)

    def call(self, u_vecs):
        if self.share_weights:
            u_hat_vecs = K.conv1d(u_vecs, self.W)
        else:
            u_hat_vecs = K.local_conv1d(u_vecs, self.W, [1], [1])
        batch_size = K.shape(u_vecs)[0]
        input_num_capsule = K.shape(u_vecs)[1]
        u_hat_vecs = K.reshape(u_hat_vecs, (batch_size, input_num_capsule,
                                            self.num_capsule, self.dim_capsule))
        u_hat_vecs = K.permute_dimensions(u_hat_vecs, (0, 2, 1, 3))
        # final u_hat_vecs.shape = [None, num_capsule, input_num_capsule, dim_capsule]

        b = K.zeros_like(u_hat_vecs[:, :, :, 0])  # shape = [None, num_capsule, input_num_capsule]
        for i in range(self.routings):
            b = K.permute_dimensions(b, (0, 2, 1))  # shape = [None, input_num_capsule, num_capsule]
            c = K.softmax(b)
            c = K.permute_dimensions(c, (0, 2, 1))
            b = K.permute_dimensions(b, (0, 2, 1))
            outputs = self.activation(K.batch_dot(c, u_hat_vecs, [2, 2]))
            if i < self.routings - 1:
                b = K.batch_dot(outputs, u_hat_vecs, [2, 3])

        return outputs

    def compute_output_shape(self, input_shape):
        return (None, self.num_capsule, self.dim_capsule)


# main.py

In [2]:
K.clear_session()

remove_stop_words = False

train_file = 'data/train.csv'
test_file = 'data/test_public.csv'
data = pd.read_csv(train_file)
test_df = pd.read_csv(test_file)

data['label'] = data['subject'] + data['sentiment_value'].astype(str)
subj_lst = list(filter(lambda x : x is not np.nan, list(set(data.label))))
subj_lst_dic = {value:key for key, value in enumerate(subj_lst)}
data['label'] = data['label'].apply(lambda x : subj_lst_dic.get(x))

data['content'] = data.content.map(lambda x: ''.join(x.strip().split()))

In [3]:
# embeddings_index
bow = BOW(data.content.apply(jieba.lcut).tolist(), min_count=1, maxlen=100)

word2vec = Word2Vec(data.content.apply(jieba.lcut).tolist(),size=300,min_count=1)

embeddings_index = {}
def get_word_embed_dict():
    for i in word2vec.wv.vocab:
        embeddings_index[i] = word2vec.wv.get_vector(i).tolist()
#     embeddings_index['UNK'] = [0]*300
    return embeddings_index
word_embed_dict = get_word_embed_dict()

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 1.149 seconds.
Prefix dict has been built succesfully.
Doc To Number: 100%|██████████| 9947/9947 [00:00<00:00, 70597.89it/s]


In [4]:
train_dict = {}
for ind, row in data.iterrows():
    content, label = row['content'], row['label']
    if train_dict.get(content) is None:
        train_dict[content] = set([label])
    else:
        train_dict[content].add(label)

In [5]:
conts = []
labels = []
for k, v in train_dict.items():
    conts.append(k)
    labels.append(v)

In [6]:
mlb = MultiLabelBinarizer()
y_train = mlb.fit_transform(labels)

In [7]:
content_list = [jieba.lcut(str(c)) for c in conts]

In [8]:
test_content_list = [jieba.lcut(c) for c in test_df.content.astype(str).values]
word_set = set([word for row in list(content_list) + list(test_content_list) for word in row])

In [9]:
word2index = {w: i + 1 for i, w in enumerate(word_set)}
seqs = [[word2index[w] for w in l] for l in content_list]
seqs_dev = [[word2index[w] for w in l] for l in test_content_list]

In [10]:
EMBEDDING_DIM= 300
embedding_matrix = np.zeros((len(word2index) + 1, EMBEDDING_DIM))
for word, i in word2index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

max_features = len(word_set) + 1

In [12]:
def get_padding_data(maxlen=100):
    x_train = sequence.pad_sequences(seqs, maxlen=maxlen)
    x_dev = sequence.pad_sequences(seqs_dev, maxlen=maxlen)
    return x_train, x_dev
maxlen = 100
X_train, X_dev = get_padding_data(maxlen)
print(X_train.shape, X_dev.shape, y_train.shape)

(8290, 100) (2364, 100) (8290, 30)


In [84]:
def get_capsule_model():
    input1 = Input(shape=(maxlen,))
    embed_layer = Embedding(len(word2index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=maxlen,
                            trainable=False)(input1)
    embed_layer = SpatialDropout1D(rate_drop_dense)(embed_layer)

    x = Bidirectional(
        GRU(gru_len, activation='relu', dropout=dropout_p, recurrent_dropout=dropout_p, return_sequences=True))(
        embed_layer)
    capsule = Capsule(num_capsule=Num_capsule, dim_capsule=Dim_capsule, routings=Routings,
                      share_weights=True)(x)
    capsule = Flatten()(capsule)
    capsule = Dropout(dropout_p)(capsule)
    output = Dense(30, activation='sigmoid')(capsule)
    model = Model(inputs=input1, outputs=output)
    model.compile(
        loss='binary_crossentropy',
        optimizer='adam',
        metrics=['accuracy'])
    return model

In [86]:
first_model_results = []
for i in range(5):
    model = get_capsule_model()
    model.fit(X_train, y_train, batch_size=64, epochs=15)
    first_model_results.append(model.predict(X_dev, batch_size=1024))
pred4 = np.average(first_model_results, axis=0)

self.W  (1, 256, 160)
u_hat_vecs before  (None, None, 160)
u_hat_vecs after1  (None, None, 10, 16)
Epoch 1/15
8290/8290 [==============================] - 26s 3ms/step - loss: 0.2309 - acc: 0.9513
Epoch 2/15
8290/8290 [==============================] - 24s 3ms/step - loss: 0.1505 - acc: 0.9603
Epoch 3/15
8290/8290 [==============================] - 24s 3ms/step - loss: 0.1406 - acc: 0.9622
Epoch 4/15
8290/8290 [==============================] - 24s 3ms/step - loss: 0.1329 - acc: 0.9635
Epoch 5/15
8290/8290 [==============================] - 24s 3ms/step - loss: 0.1278 - acc: 0.9644
Epoch 6/15
8290/8290 [==============================] - 24s 3ms/step - loss: 0.1242 - acc: 0.9647
Epoch 7/15
8290/8290 [==============================] - 24s 3ms/step - loss: 0.1205 - acc: 0.9656
Epoch 8/15
8290/8290 [==============================] - 24s 3ms/step - loss: 0.1183 - acc: 0.9658
Epoch 9/15
8290/8290 [==============================] - 24s 3ms/step - loss: 0.1161 - acc: 0.9661
Epoch 10/15
8290/82

KeyboardInterrupt: 

In [208]:
def squash(x, axis=-1):
    # s_squared_norm is really small
    # s_squared_norm = K.sum(K.square(x), axis, keepdims=True) + K.epsilon()
    # scale = K.sqrt(s_squared_norm)/ (0.5 + s_squared_norm)
    # return scale * x
    s_squared_norm = K.sum(K.square(x), axis, keepdims=True)
    scale = K.sqrt(s_squared_norm + K.epsilon())
    return x / scale

u_hat_vecs = K.ones(shape=(64,100,10,16))
u_hat_vecs = K.permute_dimensions(u_hat_vecs, (0, 2, 1, 3))
print('u_hat_vecs', K.int_shape(u_hat_vecs))
b = K.zeros_like(u_hat_vecs[:, :, :, 0])
b = K.permute_dimensions(b, (0, 2, 1))
print('b', K.int_shape(b) )
c = K.softmax(b)
c = K.permute_dimensions(c, (0, 2, 1))
b = K.permute_dimensions(b, (0, 2, 1))
print('c', K.int_shape(c))

outputs = squash(K.batch_dot(c, u_hat_vecs, [2, 2]))
print(outputs.shape)
b = K.batch_dot(outputs, u_hat_vecs, [2, 3])
print(b.shape)

u_hat_vecs (64, 10, 100, 16)
b (64, 100, 10)
c (64, 10, 100)
(64, 10, 16)
(64, 10, 100)


In [210]:
u_hat_vecs =torch.ones(64,100,10,16)
u_hat_vecs = u_hat_vecs.permute(0, 2, 1, 3)
print('u_hat_vecs', u_hat_vecs.size())
b = torch.zeros_like(u_hat_vecs[:, :, :, 0])
b = b.permute(0, 2, 1)
c = nn.functional.softmax(b)
c = c.permute(0, 2, 1)
b = b.permute(0, 2, 1)
print('c', c.size())

outputs = torch.einsum('bij,bijk->bik', (c, u_hat_vecs)) # batch matrix multiplication
print('outputs',outputs.size())

torch.einsum('bik,bijk->bij', (outputs, u_hat_vecs)).size()

u_hat_vecs torch.Size([64, 10, 100, 16])
c torch.Size([64, 10, 100])
outputs torch.Size([64, 10, 16])


/opt/algor/zhoubin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


torch.Size([64, 10, 100])

In [92]:
x = K.constant(np.array([[[1,2],[5,6],[3,9]],[[2,3],[6,7],[3,8]]]))
w = K.constant(np.array([[[0.4,0.2,1.1,4.2],[0.13,3.9,-2,-0.3]]]))
# res = K.conv1d(x,w)
res = K.local_conv1d(x,w,[1],[1])
print('x',x.shape)
print('w',w.shape)
print('res',res.shape)
x_ = x[1,:,:]
w_ = w[0,:,:]
print('x_',x_.shape)
print('w_',w_.shape)
res2 = K.dot(x_,w_)

init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    cc = sess.run(res)
    print(cc)
    print('------------------------------')
    dd = sess.run(res2)
    print(dd)

x (2, 3, 2)
w (1, 2, 4)
res (2, 1, 4)
x_ (3, 2)
w_ (2, 4)
[[[ 0.65999997  8.         -2.9         3.6       ]]

 [[ 1.19       12.1        -3.8         7.4999995 ]]]
------------------------------
[[  1.19       12.1        -3.8         7.4999995]
 [  3.31       28.500002   -7.3999996  23.099998 ]
 [  2.24       31.800001  -12.7        10.199999 ]]
